In [2]:
import numpy as np
import pickle

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding

# Load the dataset
# Load the training data
dataset = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project3/DS_5_train_input', 'rb'))
dataset_input = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project3/DS_5_train_input_prefixList', 'rb'))
dataset_output = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project3/DS_5_train_input_nextWord', 'rb'))


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Split into train and test sets
test_input = dataset_input[4500:]
test_output = dataset_output[4500:]

dataset_input = dataset_input[:4500]
dataset_output = dataset_output[:4500]

In [18]:
len(dataset_input)

25051

In [7]:
i=100

print(dataset[4500+i])
print(' '.join(test_input[i]))

a h a k b g a f b g c e b g a f c d a i a h c f b d c f b d a e b g 
a h a k b g a f b g c e b g a f c d a i a h c


In [16]:
#generating new dataset

import random
# dataset_2 = []
# dataset_output_2=[]

for sentence in dataset[:4500]:
  sent_list = sentence.split(" ")
  random_num = random.randint(1, len(sent_list) - 2)
  #print(random_num)
  dataset_input.append(sent_list[:random_num])
  dataset_output.append(sent_list[random_num])
# print(dataset[:5])
# print(dataset_2)
# print(dataset_output_2)




In [17]:
for sentence in dataset[:4500]:
  sent_list = sentence.split(" ")
  # random_start = random.randint(1, int(len(sent_list)/4))
  # random_end = random.randint(int(len(sent_list)/4), int(len(sent_list)*3/4))
  random_start = random.randint(0, len(sent_list)-2)
  random_end = random.randint(2, len(sent_list)-1)
  #print(random_num)
  subseq = sent_list[random_start:random_end]
  if len(subseq) > 0:
    dataset_input.append(sent_list[random_start:random_end])
    dataset_output.append(sent_list[random_end])

In [19]:
len(dataset_input)

25051

In [ ]:
#dataset_test = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project3/DS_6_test_input_prefixList', 'rb'))

In [ ]:
dataset_input[0]

In [20]:
input_sentences = []

for sentence in dataset_input:
  input_sentences.append(' '.join(sentence))

In [ ]:
input_sentences[0]

'a g b f a f a e a k a j c f b f c d a k'

In [21]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices(input_sentences)

batch_size = 256
batched_dataset = dataset.batch(batch_size)

In [22]:
# Preparing a TextVectorization layer

from tensorflow.keras.layers import TextVectorization

sequence_length = 200
vocab_size = 12
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(batched_dataset)

In [23]:
#Setting up a language modeling dataset

def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = batched_dataset.map(prepare_lm_dataset, num_parallel_calls=4)

In [24]:
#dividing dataset into training and validation
# Define the size of the validation set as a fraction of the total dataset
val_size = 0.25 

# Calculate the number of elements in the validation set
val_steps = int(val_size * lm_dataset.cardinality().numpy())

# Calculate the number of elements in the training set
train_steps = lm_dataset.cardinality().numpy() - val_steps

# Split the dataset into training and validation
val_dataset = lm_dataset.take(val_steps)
train_dataset = lm_dataset.skip(val_steps)


In [25]:
#A Transformer-based sequence-to-sequence model
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        #new layer 3
        # self.attention_3 = layers.MultiHeadAttention(
        #   num_heads=num_heads, key_dim=embed_dim)
        
        # self.dense_proj = keras.Sequential(
        #     [layers.Dense(dense_dim, activation="relu"),
        #      layers.Dense(embed_dim),]
        # )
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dropout(0.2),
            layers.Dense(dense_dim, activation="relu"),
            layers.Dropout(0.2),
            layers.Dense(embed_dim)
          ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        #new added
        # self.layernorm_4 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)
        #new added
        # attention_output_3 = self.attention_3(
        #     query=attention_output_2,
        #     value=encoder_outputs,
        #     key=encoder_outputs,
        #     attention_mask=padding_mask,
        # )
        # attention_output_3 = self.layernorm_3(
        #     attention_output_3 + attention_output_3)
        # proj_output = self.dense_proj(attention_output_3)
        # return self.layernorm_3(attention_output_3 + proj_output)

In [26]:
# A simple Transformer-based language model
from tensorflow import keras
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
#model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

In [27]:
#Training the sequence-to-sequence Transformer
from tensorflow.keras.optimizers import Adam

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="project3_transformer_model.keras",
        save_best_only=True,
        monitor="loss")]

model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

model.fit(train_dataset, epochs=200, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/200
74/74 [==============================] - 61s 665ms/step - loss: 1.8059 - accuracy: 0.3037 - val_loss: 1.5022 - val_accuracy: 0.3432
Epoch 2/200
74/74 [==============================] - 49s 667ms/step - loss: 1.5253 - accuracy: 0.3257 - val_loss: 1.4999 - val_accuracy: 0.3457
Epoch 3/200
74/74 [==============================] - 51s 684ms/step - loss: 1.5234 - accuracy: 0.3268 - val_loss: 1.4999 - val_accuracy: 0.3466
Epoch 4/200
74/74 [==============================] - 52s 700ms/step - loss: 1.5230 - accuracy: 0.3272 - val_loss: 1.5017 - val_accuracy: 0.3457
Epoch 5/200
74/74 [==============================] - 51s 693ms/step - loss: 1.5233 - accuracy: 0.3272 - val_loss: 1.5074 - val_accuracy: 0.3390
Epoch 6/200
74/74 [==============================] - 51s 685ms/step - loss: 1.5244 - accuracy: 0.3266 - val_loss: 1.5212 - val_accuracy: 0.3207
Epoch 7/200
74/74 [==============================] - 56s 762ms/step - loss: 1.5272 - accuracy: 0.3227 - val_loss: 1.5578 - val_accuracy:

In [28]:
import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)
    
def predict_next_word(input_sentence, temp):
  #prompt = " ".join(input_sentence)
  tokenized_sentence = text_vectorization([input_sentence])
  predictions = model.predict(tokenized_sentence, verbose=0)
  vectorized_prompt = text_vectorization([input_sentence])[0].numpy()
  prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]
  next_token = sample_next( predictions[0, prompt_length - 1 , :], temp)
  sampled_token = tokens_index[next_token]
  return sampled_token

In [ ]:
# count=0
# for temp in [0.2]:
#   count=0
#   #print("for temp :" , temp)
#   for i in range(len(test_input)):
#     #print(i)
#     input_sent = " ".join(test_input[i])
#     output = predict_next_word(input_sent, temp)
#     #print("output expected: " , test_output[i])
#     #print("output : " , output)
#     if(output == test_output[i]):
#       count+=1

#   print("with temp=", temp)
#   print(count/len(test_input))

    

# Inference on Test Dataset

In [30]:
import keras
# Loading test dataset
test_dataset = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project3/DS_6_test_input_prefixList', 'rb'))


In [35]:
# Loading the saved model

model = keras.models.load_model(
    "project3_transformer_model.keras",
    custom_objects={"TransformerDecoder": TransformerDecoder,
                    "PositionalEmbedding": PositionalEmbedding})

In [37]:
# inference on test dataset

output_data = []

for i in range(len(test_dataset)):
    input_sentence = " ".join(test_dataset[i])
    output = predict_next_word(input_sentence, 0.2)
    output_data.append(output)

pickle.dump(output_data, open('/content/drive/MyDrive/Colab Notebooks/project3/final_output_data_l1','wb'))     


In [40]:
result_data = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project3/final_output_data_l1', 'rb'))

print(result_data)

['e', '', 'e', 'c', 'c', 'd', 'b', 'a', 'b', 'f', 'a', 'a', 'b', 'c', 'a', 'k', 'g', 'a', 'd', 'd', 'a', 'g', 'e', 'd', 'k', 'a', 'a', 'c', 'c', 'i', '', 'c', 'b', 'g', 'e', 'h', 'g', 'b', 'k', 'a', 'a', 'c', 'd', 'g', '[UNK]', 'b', 'g', 'a', 'g', 'f', '[UNK]', 'i', '', 'g', 'c', 'a', 'd', 'f', 'c', 'a', 'g', 'b', 'c', 'd', 'c', 'f', 'a', 'b', 'b', 'c', 'g', 'a', 'a', 'a', 'c', 'a', 'd', '', 'e', 'b', 'k', 'f', 'b', 'b', 'e', 'b', 'a', 'c', '', 'c', 'e', 'b', 'e', 'g', 'd', 'g', 'a', 'c', 'b', 'i', 'a', 'c', 'a', 'd', 'c', 'a', 'd', 'a', 'c', 'f', 'f', 'f', 'd', 'g', 'a', 'h', 'k', 'a', 'e', 'k', 'a', 'e', 'c', 'f', 'a', '', 'f', 'c', '', 'g', 'b', '[UNK]', 'c', 'f', 'd', 'k', 'a', 'a', 'h', 'f', 'b', 'a', 'd', 'b', 'a', 'g', 'b', 'd', 'g', 'd', 'b', 'a', 'b', 'e', 'e', 'a', 'g', 'e', 'f', 'c', '[UNK]', 'b', '[UNK]', 'f', 'b', 'b', 'f', 'a', 'b', 'g', 'a', 'a', 'c', 'a', 'e', 'a', 'c', '[UNK]', 'k', 'a', 'a', 'd', 'd', 'e', 'e', 'f', 'g', 'b', 'g', 'i', '', 'g', '', 'd', 'g', 'e', 'd',